<a href="https://colab.research.google.com/github/sysbio-unam/GAFerm/blob/main/Copy_of_Opt_ML_STUDENTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authors:
## Antonio del Rio Chanona      https://www.imperial.ac.uk/people/a.del-rio-chanona
## Edgar Ivan Sanchez Medina    https://www.mpi-magdeburg.mpg.de/person/103552/2316

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

Along this notebook you will find <font color='blue'>text in blue that describe the **coding tasks** that you have to implement.</font> Within the code cell, your implementation needs to be between the line:

`#-#-#-#-#-#-#-#-#-# Start of your code #-#-#-#-#-#-#-#-#-#`

and the line:

`#-#-#-#-#-#-#-#-#-# End of your code #-#-#-#-#-#-#-#-#-#-#`

# **1. Optimization basics**

An optimization problem has the form

\begin{equation}
\begin{aligned}
\min_{x \in X} \quad & f(x)\\
\textrm{s.t.} \quad & g_i(x) \leq 0, i= 1, ..., m\\
\end{aligned}
\end{equation}

The vector $x = (x_1,..., x_n)$ is the optimization variable of the problem, the function $f : \mathbb{R}^n \to \mathbb{R} $ is the objective function, the functions $g_i : \mathbb{R}^n \to \mathbb{R} $,
$i = 1, ...,m$, are the (inequality) constraint functions. A vector $x^⋆$ is called optimal, if it has the smallest objective value among all vectors
that satisfy the constraints.

**Convex optimization**

A convex optimization problem is one in which the objective and the
constraint functions are convex, which means they satisfy the inequality:

\begin{equation}
f(\alpha x + \beta y) \leq \alpha f(x) + \beta f(y)
\end{equation}

for all $x,y \in \mathbb{R}^n$ and all $\alpha, \beta \in \mathbb{R}$ with $\alpha + \beta = 1$, $\alpha  \geq 0$ and $\beta  \geq 0$

It is worth nothing that any equality constraint can be represented as two inequality constraints. This enforce that, in a convex problem, the equality constraints must be linear.

Therefore, an optimization problem is convex whenever the following requirements are met:

* The objective function is convex.
* The inequality constraints are convex.
* The equality constraints are linear.

**Global and local optima**

An optimal solution $x^*$ is said to be the global optimum when the constraints are met at this point and $f(x^*) \leq f(x),  \forall x \in X$. When this condition is met only whitin a certain neightborhood, the solution is called local optimum.




In [ ]:
def Rosenbrock_f(x):
    '''
    Rosenbrock function
    '''
    n = np.shape(x)[1]
    z = np.sum(100*(x[:,1:] - x[:,:n-1]**2)**2 + (x[:,:n-1] - 1)**2, axis=1)
    return z

###############################################
# --- Central finite differences 5 points --- # 
###############################################

def central_finite_diff5(f, x):
      '''
      Five-points method for central finite differences.
      INPUTS:
          f  : Function
          x  : Position where to approximate the gradient
      OUTPUTS:
          grad: Approximation of the gradient of f at x 
      '''
      dim = x.shape[1]
      # Step-size is taken as the square root of the machine precision
      eps  = np.sqrt(np.finfo(float).eps) 
      grad = np.zeros((1,dim))
        
      for i in range(dim):
          e           = np.zeros((1,dim))
          e[0,i]      = eps
          grad_approx = (f(x - 2*e) - 8*f(x - e) + 8*f(x + e) - f(x + 2*e) )/(12*eps) 
          
          grad[0,i]     = grad_approx
        
      return grad

# **10. Newton's method**

This optimization method calculates the inverse of the Hessian matrix to obtain faster convergence than the first-order gradient descent methods.

The update rule of the Newton's method is:

$ x^{(k+1)} = x^{(k)} - \left(\nabla^2 f(x^{(k)}) \right)^{-1}\nabla f(x^{(k)}) $

The second term from the right hand side of the above equation is then the Newton step, and it requires, from the inverse of the Hessian, for the Hessian matrix to be positive definite. However, for some non-convex problems the Hessian might not be invertible.

Similarly as what we did for approximating the gradient using central finite differences, we can approximate the Hessian. For the details and formulas of the Hessian approximation look [here](https://en.wikipedia.org/wiki/Finite_difference).

In [ ]:
###################################################
# --- Central second order finite differences --- #
###################################################

def Second_diff_fxx(f, x):
    '''
      Central finite differences approximation of Hessian
      INPUTS:
          f  : Function
          x  : Position where to approximate the Hessian
      OUTPUTS:
          Hxx: Approximation of the Hessian of f at x 
      '''

    dim   = np.shape(x)[1]
    # Step-size is taken as the square root of the machine precision
    eps  = np.sqrt(np.finfo(float).eps)
    Hxx   = np.zeros((dim,dim))
    
    for j in range(dim):
        # compute Fxx (diagonal elements)
        x_d_f       = np.copy(x)             # forward step
        x_d_b       = np.copy(x)             # backward step
        x_d_f[0,j]  = x_d_f[0,j] + eps
        x_d_b[0,j]  = x_d_b[0,j] - eps
        Hxx[j,j]    = (f(x_d_f) -2*f(x) + f(x_d_b))/eps**2

        for i in range(j+1,dim):
            # compute Fxy (off-diagonal elements)
            # Fxy
            x_d_fxfy    = np.copy(x_d_f)
            x_d_fxfy[0,i] = x_d_fxfy[0,i] + eps
            x_d_fxby    = np.copy(x_d_f)
            x_d_fxby[0,i] = x_d_fxby[0,i] - eps
            x_d_bxfy    = np.copy(x_d_b)
            x_d_bxfy[0,i] = x_d_bxfy[0,i] + eps
            x_d_bxby    = np.copy(x_d_b)
            x_d_bxby[0,i] = x_d_bxby[0,i] - eps
            Hxx[j,i]    = (f(x_d_fxfy) - f(x_d_fxby) - f(x_d_bxfy) + f(x_d_bxby))/(4*eps**2)
            Hxx[i,j]    = Hxx[j,i]

    return Hxx


x1       = np.array([2.,2.]).reshape(1,-1)
Hxx = Second_diff_fxx(Rosenbrock_f, x1)
print('Hxx = ',Hxx)

In [ ]:
###########################
# --- Newton's Method --- #
###########################

def newton(f, x0, grad_f, H_f, max_iter=1e3, grad_tol=1e-4, traj=False):
    '''
    Newton's method

    Note: the Hessian can become ill-conditioned
    INPUTS:
        f        : Function
        x0       : Initial guess
        grad_f   : Gradient function
        H_f      : Hessian function
        max_iter : Maximum number of iterations
        grad_tol : Tolerance for gradient approximation
        traj     : Boolean for plotting
    OUTPUTS:
        x        : Optimal point
        iter_i   : Number of iterations needed
    '''
    
    # initialize problem
    n      = np.shape(x0)[0]
    x      = np.copy(x0)
    iter_i = 0
    grad_i = grad_tol*10
    
    # trajectory
    if traj == True:
            x_list = []
            f_list = []            
    
    # optimization loop
    while np.sum(np.abs(grad_i)) > grad_tol and iter_i < max_iter: 
              
        #-#-#-#-#-#-#-#-#-# Start of your code #-#-#-#-#-#-#-#-#-#      
        grad_i  = 00000                         # compute gradient
        Hxx     = 00000                            # compute Hessian
        x       = 00000 # update  
        #-#-#-#-#-#-#-#-#-# End of your code #-#-#-#-#-#-#-#-#-#            
        iter_i += 1
        
        # trajectory
        if traj == True:
            x_list.append(x.flatten().tolist())
            f_list.append(f(x))
    
    print(" Optimization using Newton's method \n")
    print('Iterations: ', iter_i)
    print('Optimal x : ', x) 
    print('Final grad: ', grad_i)

    # trajectory    
    if traj == True:
        return x, x_list, f_list
        
    return x, iter_i

In [ ]:
# --- Newton's method --- #
x0 = np.array([0.,0.]).reshape(1,-1)

xf, x_list, f_list = newton(Rosenbrock_f, x0, central_finite_diff5, Second_diff_fxx, traj=True)

In [ ]:
x_array = np.array(x_list).reshape(-1,2)
fig  = plt.figure()
cs   = plt.scatter(x_array[:,0], x_array[:,1], marker=".", c=f_list , cmap="seismic")
cbar = fig.colorbar(cs)
plt.xlabel(r'$x_1$'); plt.ylabel(r'$x_2$'); plt.title("Rosenbrock with Newton's method")
plt.show()

Of course, using the Hessian information accelerates the optimization significantly! But, this comes at the expense of not only obtaining the Hessian, but also inverting it! This is the reason why second-order methods are not used in practice with large datasets. But, if the dataset is relatively small, go ahead and use second-order methods (whenever you have access to the second derivatives, of course).

# **11. Constrained Newton's method**

If we recall the Newton's method, the step (called Newton step) that we take at each iteration is:

\begin{equation}
\Delta x = - \left(\nabla^2 f(x^{(k)}) \right)^{-1}\nabla f(x^{(k)})
\end{equation}

A nice interpretation of the Newton’s step is to set
it by minimizing the second-order approximation of $f$ at $x$.
In case of (equality) constrained problems, the Newton's method is slightly modified in two main aspects:

* The initial point must be feasible.
* The definition of Newton step is modified to make sure that the newton step is a feasible direction of the problem.

To derive the Constrained Newton's method the objective function in the original problem:

\begin{equation}
\begin{aligned}
\min_{x \in X} \quad & f(x)\\
\textrm{s.t.} \quad & h_i(x) = 0, i= 1, ..., m\\
\end{aligned}
\end{equation}

is approximated by the second-order Taylor expansion:

\begin{equation}
\begin{aligned}
\min_{\Delta x \in X} \quad & \hat{f}(x + \Delta x) = f(x) + \nabla f(x)^T \Delta x + \frac{1}{2} \Delta x ^T \nabla^2 f(x) \Delta x\\
\textrm{s.t.} \quad & h_i(x + \Delta x) = 0, i= 1, ..., m\\
\end{aligned}
\end{equation}

The Newton step $\Delta x$ now can be obtained by solving the following system of equations: 


\begin{gather}
 \begin{bmatrix} 
 \nabla^2 f(x) & \nabla h(x)^T \\ 
 \nabla h(x) & 0 
 \end{bmatrix}
 \begin{bmatrix} 
 \Delta x \\ 
 \lambda 
 \end{bmatrix}
 =
 -
 \begin{bmatrix} 
 \nabla f(x) \\ 
 h(x) 
 \end{bmatrix}
\end{gather}





In [ ]:
##############################
# --- Objective function --- #
##############################

def Obj(x):
  '''
  Objective function
  '''
  return x[:,0]**2 + 5*x[:,1]**2

##################################
# --- Gradient Obj. function --- #
##################################

def gradient(x):
  '''
  Gradient of objective function
  '''
  g1 = 2*x[:,0]
  g2 = 10*x[:,1]
  return  np.array([g1,g2])

###################
# --- Hessian --- #
###################
exact_hess = np.array([[2,0],[0, 10]]) 

################################
# --- Equality constrained --- #
################################

def Const(x):
  '''
  Equality constrained
  '''
  return -x[:,0] - x[:,1] - 2

#########################
# --- Plot function --- #
#########################

x_1 = np.linspace(-4,4)
x_2 = np.linspace(-2,2)
x = np.vstack((x_1,x_2)).reshape(-1,2)
X, Y = np.meshgrid(x_1, x_2)
Z = Obj(np.append(X.reshape(-1,1), Y.reshape(-1,1), axis=1))
Z = Z.reshape(X.shape)

const_val = Const(x)

fig = plt.figure(figsize=(6,5))
left, bottom, width, height = 0.15, 0.1, 0.8, 0.8
ax = fig.add_axes([left, bottom, width, height]) 
contours = ax.contour(x_1, x_2, Z, colors='black', alpha=0.8)
ax.clabel(contours, inline=True, fontsize=8)
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
plt.xlim(-4, 4)
plt.ylim(-2, 2)
plt.plot(x_1, -x_1-2, 'b-',label='h(x)=0')
plt.legend()



In [ ]:
# Using Scipy SLSQP to see what to expect

from scipy.optimize import minimize

def obj_f(x):
  return x[0]**2 + 5*x[1]**2

initial = np.array([-2, 0])
cons = ({'type': 'eq', 'fun': lambda x:  -x[0] -x[1] - 2})

res = minimize(obj_f, initial, method='SLSQP', tol=1e-15, constraints=cons)
print(res.fun)
print(res.x)

In [ ]:
###################################
# --- Constrained Newton step --- #
###################################

def Const_Newton_step(H, J, g, h):
  '''
  Determine the constrained Newton step

    INPUTS:
      H: Hessian matrix of the objective function at x
      J: Jacobian of constraints at x
      g: Gradient of objective function at x
      h: Constraints value at x
        
    OUTPUTS:
      step: Constrained Newton step
      lambda: Lagrange multipliers
  '''
  dim = H.shape[0]
  num_const = J.shape[0]
  dim_lag = dim + num_const

  A = np.zeros((dim_lag, dim_lag))
  A[:dim,:dim] = H; 
  A[dim:,:dim][0] = J; 
  A[:dim,dim:][:,0] = J.T.reshape(dim,)
  B     = -np.append(g, h)
  sol   = np.linalg.solve(A, B)
  step  = sol[:dim]
  lamda = sol[dim:]

  return step, lamda

In [ ]:
#######################################
# --- Constrained Newton's Method --- #
#######################################

def const_newton(f, const, x0, grad_f, H_f, max_iter=1e3, tol=1e-4, traj=False):
    '''
    Constrained Newton's method

    INPUTS:
        f        : Function
        const    : Constraint
        x0       : Initial guess
        grad_f   : Gradient function
        H_f      : Hessian function
        max_iter : Maximum number of iterations
        grad_tol : Tolerance for gradient approximation
        traj     : Boolean for plotting
    OUTPUTS:
        x        : Optimal point
        iter_i   : Number of iterations needed
    '''
    
    # initialize problem
    x      = np.copy(x0)
    iter_i = 0
    decr   = 10
    
    # trajectory
    if traj == True:
            x_list = []
            f_list = []            
    
    # optimization loop
    while decr/2 > tol and iter_i < max_iter:
        #H       = H_f(f, x)                             # compute Hessian of Obj function

        #-#-#-#-#-#-#-#-#-# Start of your code #-#-#-#-#-#-#-#-#-# 
        H       = 00000
        J       = 00000     # compute Jacobian of constraint
        g       = 00000                             # compute gradient of Obj function
        h       = 00000                              # compute constraint value                     
        delta_x, lamb = 00000      # compute step and lambda
        x       = 00000                           # update 
        decr    = 00000               # decrement  
        #-#-#-#-#-#-#-#-#-# End of your code #-#-#-#-#-#-#-#-#-#   
                
        iter_i += 1
        
        # trajectory
        if traj == True:
            x_list.append(x.flatten().tolist())
            f_list.append(f(x))
    
    print(" Optimization using constrained Newton's method \n")
    print('Iterations: ', iter_i)
    print('Optimal x : ', x) 

    # trajectory    
    if traj == True:
        return x, x_list, f_list
        
    return x, iter_i

In [ ]:
# --- Constrained Newton's method --- #
x0 = np.array([3,1.5]).reshape(1,-1)

xf, x_list, f_list = const_newton(Obj, Const, x0, gradient, Second_diff_fxx, traj=True)

In [ ]:
fig = plt.figure(figsize=(6,5))
left, bottom, width, height = 0.15, 0.1, 0.8, 0.8
ax = fig.add_axes([left, bottom, width, height]) 
contours = ax.contour(x_1, x_2, Z, colors='black', alpha=0.8)
ax.clabel(contours, inline=True, fontsize=8)
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
plt.xlim(-4, 4)
plt.ylim(-2, 2)
plt.plot(x_1, -x_1-2, 'b-',label='h(x)=0')
plt.plot(x0[:,0], x0[:,1], 'ro', label='initial point')
plt.plot(x_list[1][0], x_list[1][1], 'r*', label='Optimum', ms=10)
plt.legend()

# **12. Barrier method (interior-point methods)**

With the Constrained Newton's method we accomplished to solve optimization problem with equality constraints. With interior-point methods we can solve convex problems that include inequality constraints. Interior-point methods solve this type of problems by solving a sequence of equality constrained problems using Newton's method. The barrier method is one of these interior-point algorithms, and is the one we are going to focus here.

The goal is to approximately formulate the inequality constrained problem 

\begin{equation}
\begin{aligned}
\min_{x \in X} \quad & f(x)\\
\textrm{s.t.} \quad & h_i(x) = 0, i= 1, ..., m\\
\quad & g_j(x) \leq 0, j=1,...,n
\end{aligned}
\end{equation}


as an equality constrained problem to which Newton’s method can be applied. This is accomplished by making the inequality constraints $g_j(x)$ implicit in the objective function $f(x)$.

\begin{equation}
\begin{aligned}
\min_{x \in X} \quad & f(x) + \sum_{j=1}^{n}I_-(g_j(x))\\
\textrm{s.t.} \quad & h_i(x) = 0, i= 1, ..., m\\
\end{aligned}
\end{equation}

where $I_-$ is the indicator function for the nonpositive reals:

\begin{equation}
\begin{aligned}
    I_-(u)= 
\begin{cases}
    0, &  u \leq 0\\
    \infty,    & u > 0
\end{cases}
\end{aligned}
\end{equation}

The problem with this formulation is that, even when we accomplished to make the inequality constraints implicit in the objective function, this objective function is not (in general) differentiable, so Newton's method cannot be applied. The basic idea of the barrier method is to approximate this indicator function as:

\begin{equation}
    \hat{I}_-(u) = - \frac{1}{t} \log (-u) 
\end{equation}

Therefore, the approximated problem that is going to be solved is:

\begin{equation}
\begin{aligned}
\min_{x \in X} \quad & f(x) + \sum_{j=1}^{n}- \frac{1}{t} \log (-g_j(x))\\
\textrm{s.t.} \quad & h_i(x) = 0, i= 1, ..., m\\
\end{aligned}
\end{equation}

where the function $\phi(x) = - \sum_{j=1}^{n} \log (-g_j(x))$ is called the *logarithmic barrier* for the problem.

The accuracy of this approximation improves as the parameter $t$ increases. However, when the parameter $t$ is large, the objective function
is difficult to minimize by Newton’s method, since its Hessian varies rapidly near the boundary of the feasible set. This problem can be overcomed by solving a sequence of problems of the same form, increasing the parameter $t$ (and therefore the accuracy of the approximation) at each step, and starting each Newton minimization at the solution of the problem for the previous value of t.

The gradient and the Hessian of the logarithmic barrier function are given by:

\begin{equation}
    \nabla \phi (x) = \sum_{j=1}^{n} - \frac{1}{g_j(x)} \nabla g_j(x)
\end{equation}

\begin{equation}
    \nabla^2 \phi (x) = \sum_{j=1}^{n} \frac{1}{g_j(x)^2} \nabla g_j(x) \nabla g_j(x)^T - \sum_{j=1}^{n} \frac{1}{g_j(x)} \nabla^2 g_j(x)
\end{equation}


**Algorithm**

1. Given a strictly feasible starting point $x \in X$, $t := t^{(0)}$ $\alpha > 0$, $\epsilon >0$.
2. Repeat until $n/t \leq \epsilon$
3. $~~~~~~$ Solve approximated problem with $t:= t^{(k)}$ starting from $x^{k}$
2. $~~~~~~$ Update $x^{k+1} := x^{*(k)}$
3. $~~~~~~$ Increase $t^{k+1} := \alpha t^{(k)}$

The example problem to solve is:

\begin{equation}
\begin{aligned}
\min_{x_1, x_2 \in X} \quad & x_1^2 + 5x_2^2\\
\textrm{s.t.} \quad & -x_1-x_2-2 = 0\\
& -x_1-x_2 \leq 0
\end{aligned}
\end{equation}

In [ ]:
# Inequality constraint
def Ineq_const(x):
  '''
  Inequality constraint
  '''
  return -x[:,0] + x[:,1]

grad_ineq = np.array([-1, 1])           # Gradient of inequality constraint
hess_ineq = np.array([[0,0], [0,0]])    # Hessian of inequality constraint

# Plot
x = np.vstack((x_1,x_2)).reshape(-1,2)
ineq_const_val = Ineq_const(x)
fig = plt.figure(figsize=(6,5))
left, bottom, width, height = 0.15, 0.1, 0.8, 0.8
ax = fig.add_axes([left, bottom, width, height]) 
contours = ax.contour(x_1, x_2, Z, colors='black', alpha=0.8)
ax.clabel(contours, inline=True, fontsize=8)
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
plt.xlim(-4, 4)
plt.ylim(-2, 2)
plt.plot(x_1, -x_1-2, 'b-',label='h(x)=0')
plt.plot(x_1, x_1, 'b--',label='g(x)=0')
plt.fill_between(x_1, x_1, np.repeat(4, len(x_1)), color='k', alpha=0.3)
plt.legend()

In [ ]:
##########################################
# --- Approximation with log barrier --- #
##########################################

def Obj_barrier(x):
  '''
  Objective function with log barrier
  '''
  t = 0.1
  phi = - np.log(-Ineq_const(x))
  return x[:,0]**2 + (5*x[:,1])**2 + 1/t*phi


Z2 = Obj_barrier(np.append(X.reshape(-1,1), Y.reshape(-1,1), axis=1))
Z2 = Z2.reshape(X.shape)

const_val = Const(x)
x = np.vstack((x_1,x_2)).reshape(-1,2)
ineq_const_val = Ineq_const(x)

# Hacer subplots
fig = plt.figure(figsize=(6,5))
left, bottom, width, height = 0.15, 0.1, 0.8, 0.8
ax = fig.add_axes([left, bottom, width, height]) 
contours = ax.contour(x_1, x_2, Z, colors='black', alpha=0.8)
plt.contour(x_1, x_2, Z2, colors='green', alpha=0.8)
ax.clabel(contours, inline=True, fontsize=8)
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
plt.xlim(-4, 4)
plt.ylim(-2, 2)
plt.plot(x_1, -x_1-2, 'b-',label='h(x)=0')
plt.plot(x_1, x_1, 'b--',label='g(x)=0')
plt.fill_between(x_1, x_1, np.repeat(4, len(x_1)), color='k', alpha=0.3)
plt.legend()

In [ ]:
# Using Scipy SLSQP to see what to expect

from scipy.optimize import minimize

def obj_f(x):
  return x[0]**2 + 5*x[1]**2

initial = np.array([-2, 0])
cons = ({'type': 'eq', 'fun': lambda x:  -x[0] -x[1] - 2},
        {'type': 'ineq', 'fun': lambda x:  x[0] -x[1]})

res = minimize(obj_f, initial, method='SLSQP', tol=1e-15, constraints=cons)
print(res.fun)
print(res.x)

In [ ]:
#######################################
# --- Hessian logarithmic barrier --- #
#######################################

def hess_logbarrier(x):
  '''
  Hessian of the logarithmic barrier function
  '''
  val_ineq = Ineq_const(x)[0]
  h1 = 1/(val_ineq**2) * grad_ineq.reshape(-1,1) @ grad_ineq.reshape(-1,1).T 
  h2 = - 1/(val_ineq)*hess_ineq
  return h1 + h2

########################################
# --- Gradient logarithmic barrier --- #
########################################

def grad_logbarrier(x):
  '''
  Gradient of the logarithmic barrier function
  '''
  val_ineq = Ineq_const(x)[0]
  return -1/val_ineq*grad_ineq


In [ ]:
###################################
# --- Constrained Newton step --- #
###################################

def DeltaX(H, J, g, h):
  '''
  Determine the constrained Newton step

    INPUTS:
      H: Hessian matrix of the objective function at x
      J: Jacobian of constraints at x
      g: Gradient of objective function at x
      h: Constraints value at x
        
    OUTPUTS:
      step: Constrained Newton step
      lambda: Lagrange multipliers
  '''
  dim = H.shape[0]
  num_const = J.shape[0]
  dim_lag = dim + num_const

  A = np.zeros((dim_lag, dim_lag))
  A[:dim,:dim] = H; 
  A[dim:,:dim][0] = J; 
  A[:dim,dim:][:,0] = J.T.reshape(dim,)
  B     = -np.append(g, h)
  sol   = np.linalg.solve(A,B)
  step  = sol[:dim]
  lamda = sol[dim:]

  return step, lamda

In [ ]:
##########################
# --- Barrier method --- #
##########################

def barrier(f, eq_const, x0, grad_f, t, alpha=1.01, max_iter=1e4, tol=1e-4, traj=False):
    '''
    Barrier method (interior-point methods)

    INPUTS:
        f        : Function
        eq_const : Equality constraint
        x0       : Initial guess
        grad_f   : Gradient function
        t        : Barrier parameter
        max_iter : Maximum number of iterations
        grad_tol : Tolerance for gradient approximation
        traj     : Boolean for plotting
    OUTPUTS:
        x        : Optimal point
        iter_i   : Number of iterations needed
    '''
    
    # initialize problem
    x      = np.copy(x0)
    iter_i = 0
    n_iq   = 1             # number of inequality constraints
    
    # trajectory
    if traj == True:
            x_list = []
            f_list = []            

    # optimization loop
    while n_iq/t >= tol*0.001 and iter_i < max_iter: 
      decr = 10
      while decr/2 > tol and iter_i < max_iter:
        J   = np.array([-1, -1]).reshape(1,-1)     # compute Jacobian of constraint
        h   = eq_const(x)                          # compute constraint value
        
        x_1 = x[:,0][0]
        x_2 = x[:,1][0]
        g = np.array([2*x_1 - 1/t*x_1/(x_1-x_2), 10*x_2 + 1/t*x_2/(x_1-x_2)]).reshape(-1,1) 
        
        h1 = 2 + 1/t*x_2/(x_1 - x_2)**2
        h2 = -1/t*x_1/(x_1 - x_2)**2
        h3 = -1/t*x_2/(x_1 - x_2)**2
        h4 = 10 + 1/t*x_1/(x_1 - x_2)**2
        H_f = np.array([[h1, h2], [h3, h4]])                
        
        #delta_x = -np.matmul(np.linalg.pinv(H_f),g) 
        #delta_x, la = Const_Newton_step(H_f,J,g,h)      # compute step and lambda
        delta_x, la = DeltaX(H_f,J,g,h)
        x       = x + delta_x                             # update x
        decr    = delta_x.T @ H_f @ delta_x               # decrement
        iter_i += 1
      t       = t * alpha                             # update t
      # trajectory
      if traj == True:
          x_list.append(x.flatten().tolist())
          f_list.append(f(x))
    
    print(" Optimization using barrier method \n")
    print('Iterations: ', iter_i)
    print('Optimal x : ', x) 

    # trajectory    
    if traj == True:
        return x, x_list, f_list
        
    return x, iter_i

In [ ]:
# --- Barrier method --- #
x0 = np.array([3.5,1.5]).reshape(1,-1)
t0 = 0.0001

xf, x_list, f_list = barrier(Obj, Const, x0, gradient, t0, traj=True)

In [ ]:
fig = plt.figure(figsize=(6,5))
left, bottom, width, height = 0.15, 0.1, 0.8, 0.8
ax = fig.add_axes([left, bottom, width, height]) 
contours = ax.contour(x_1, x_2, Z, colors='black', alpha=0.8)
ax.clabel(contours, inline=True, fontsize=8)
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
plt.xlim(-4, 4)
plt.ylim(-2, 2)
plt.plot(x0[:,0], x0[:,1], 'ro', label='initial point')
plt.plot(x_list[-1][0], x_list[-1][1], 'r*', label='Optimum', ms=10)
plt.plot(x_1, -x_1-2, 'b-',label='h(x)=0')
plt.plot(x_1, x_1, 'b--',label='g(x)=0')
plt.fill_between(x_1, x_1, np.repeat(4, len(x_1)), color='k', alpha=0.3)
plt.legend()